# 🔍 Advanced RAG Patterns - Interactive Demo

**Welcome to Advanced RAG techniques!**

In Notebook 1, we built a basic RAG system that retrieves documents and generates answers. While that works for simple use cases, production RAG systems need more sophisticated techniques to handle:

- **Large document collections** (thousands or millions of documents)
- **Complex queries** requiring multi-step reasoning
- **Poor retrieval results** that need improvement
- **Answer quality** and accuracy verification

This demo shows you **production-grade RAG patterns** used by leading AI companies.

## 📚 What This Demo Covers

### **1. Query Rewriting & Expansion**
Improve retrieval by reformulating user queries into better search terms.

### **2. Hybrid Search (Dense + Sparse)**
Combine semantic similarity (embeddings) with keyword matching (BM25) for better results.

### **3. Reranking**
Use a specialized model to reorder retrieved documents by relevance.

### **4. RAG Fusion**
Generate multiple query variations, retrieve for each, and combine results.

### **5. Contextual Compression**
Extract only the relevant portions from retrieved documents to reduce noise.

### **6. Self-RAG (Self-Reflective RAG)**
Agent evaluates its own answers and decides whether to retrieve more information.

### **7. CRAG (Corrective RAG)**
Automatically detects poor retrievals and falls back to web search.

---

**Each pattern includes:**
- Why you need it (the problem it solves)
- How it works (the technique)
- Working code example
- When to use it in production

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## 1) Query Rewriting

**The Problem:**
Users often ask questions in ambiguous or poorly-worded ways. If you search for exactly what they typed, you might miss relevant documents.

**Example:**
- User asks: *"How do I make my app faster?"*
- Better query: *"application performance optimization techniques"*

**The Solution:**
Use an LLM to rewrite the user's query into a more effective search query before retrieving documents.

### Code Context
We create a chain that:
1. Takes the original user query
2. Asks the LLM to rewrite it for better retrieval
3. Uses the rewritten query to search

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Query rewriter
query_rewrite_template = """You are an expert at reformulating user queries for better document retrieval.

Original query: {question}

Rewrite this query to be more specific and effective for semantic search. 
Focus on key concepts and technical terms.

Rewritten query:


"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
query_rewriter = (
    ChatPromptTemplate.from_template(query_rewrite_template)
    | llm
    | StrOutputParser()
)

# Example
original_query = "How do I make my app faster?"
rewritten_query = query_rewriter.invoke({"question": original_query})

print(f"Original: {original_query}")
print(f"Rewritten: {rewritten_query}")

## 2) Multi-Query Generation (RAG Fusion)

**The Problem:**
A single query might not capture all aspects of what the user wants. Different phrasings retrieve different documents.

**The Solution:**
Generate multiple variations of the query, retrieve documents for each, and combine the results with ranking fusion.

### Code Context
This technique:
1. Generates 3-5 different query variations
2. Retrieves documents for each variation
3. Uses Reciprocal Rank Fusion to combine results

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

multi_query_template = """You are an AI assistant helping to improve search results.

Generate 3 different versions of the given question to retrieve relevant documents from a vector database.
Provide these alternative questions separated by newlines.

Original question: {question}

Alternative questions:"""

multi_query_generator = (
    ChatPromptTemplate.from_template(multi_query_template)
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

# Example
question = "What are the benefits of using LangChain?"
alternative_queries = multi_query_generator.invoke({"question": question})

print("Generated queries:")
for i, q in enumerate(alternative_queries, 1):
    if q.strip():
        print(f"{i}. {q.strip()}")

## 3) Contextual Compression

**The Problem:**
Retrieved documents are often long and contain irrelevant information. Sending entire documents to the LLM:
- Wastes tokens (costs money)
- Adds noise (reduces accuracy)
- Hits context limits

**The Solution:**
Use a small, fast model to extract only the relevant portions of each document before sending to the main LLM.

### Code Context
We use LangChain's `ContextualCompressionRetriever` with an `LLMChainExtractor` to:
1. Retrieve documents normally
2. Extract only relevant passages
3. Return compressed results

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Load and prepare documents
loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = splitter.split_documents(docs)

# Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = InMemoryVectorStore(embeddings)
vectorstore.add_documents(splits)

# Base retriever
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Add compression
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# Test it
question = "What is LangChain?"
compressed_docs = compression_retriever.invoke(question)

print(f"Retrieved {len(compressed_docs)} compressed documents")
print("\nFirst compressed document:")
print(compressed_docs[0].page_content[:300] + "...")

## 4) Self-Querying Retriever

**The Problem:**
Users often include filters in their natural language query:
- *"Find articles about Python published after 2023"*
- *"Show me budget hotels in Paris"*

Basic semantic search doesn't handle these filters well.

**The Solution:**
Use an LLM to parse the query into:
1. Semantic search query ("Python articles")
2. Metadata filters (year > 2023)

### Code Context
The self-querying retriever automatically:
- Extracts filter criteria from natural language
- Applies them to the vector search

In [ ]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# Define metadata fields your documents have
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The source URL of the document",
        type="string",
    ),
    AttributeInfo(
        name="title",
        description="The title of the document",
        type="string",
    ),
]

document_content_description = "LangChain documentation and tutorials"

# Create self-querying retriever
self_query_retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=True
)

# Example with implicit filter
query = "What is LangChain? Only show me content from the introduction page"
results = self_query_retriever.invoke(query)

print(f"Found {len(results)} results")

## 5) Ensemble Retrieval (Hybrid Search)

**The Problem:**
- **Semantic search** (embeddings) is great for concept matching but can miss exact keyword matches
- **Keyword search** (BM25) finds exact terms but misses semantic similarity

**Example:**
- Query: *"What is AI?"*
- Semantic search finds "artificial intelligence, machine learning" ✅
- Keyword search only finds documents with exact phrase "What is AI" ❌

**The Solution:**
Combine both approaches and use Reciprocal Rank Fusion to merge results.

### Code Context
The ensemble retriever:
1. Runs both BM25 (keyword) and semantic search in parallel
2. Merges results using weighted fusion

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

# Create BM25 retriever (keyword-based)
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 5

# We already have semantic retriever (base_retriever)
semantic_retriever = base_retriever

# Combine them with ensemble
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, semantic_retriever],
    weights=[0.5, 0.5]  # Equal weight to both
)

# Test
query = "LangChain agents"
ensemble_results = ensemble_retriever.invoke(query)

print(f"Ensemble retrieval found {len(ensemble_results)} results")
print("\nTop result:")
print(ensemble_results[0].page_content[:200] + "...")

## 6) Parent Document Retriever

**The Problem:**
When you split documents into small chunks for embedding:
- Small chunks are great for precise retrieval
- But they lack surrounding context when generating answers

**The Solution:**
Store small chunks for retrieval, but return the full parent document (or larger chunk) for generation.

### Code Context
- Index: Small chunks (100-200 tokens)
- Retrieve: Find relevant small chunks
- Return: The full parent documents for context

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

# Parent splitter (large chunks)
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

# Child splitter (small chunks for retrieval)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# Store for parent documents
store = InMemoryStore()

# Create parent document retriever
parent_doc_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

# Add documents
parent_doc_retriever.add_documents(docs)

# Retrieve
results = parent_doc_retriever.invoke("What is LangChain?")

print(f"Retrieved {len(results)} parent documents")
print(f"\nFirst parent doc length: {len(results[0].page_content)} characters")

## 7) CRAG (Corrective RAG)

**The Problem:**
Sometimes your document collection simply doesn't contain the answer. The retriever returns irrelevant documents, and the LLM generates a hallucinated answer.

**The Solution:**
Build an agent that:
1. Retrieves documents from your knowledge base
2. **Evaluates** if they're actually relevant
3. If yes → generates answer from documents
4. If no → searches the web instead

### Code Context
We use LangGraph to build a corrective RAG agent with:
- Retrieval grading (are docs relevant?)
- Web search fallback
- Hallucination checking

In [ ]:
from typing import TypedDict, List
from langchain_core.documents import Document
from pydantic import BaseModel, Field

# Grader for evaluating document relevance
class GradeDocuments(BaseModel):
    """Binary score for relevance check."""
    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

# LLM with structured output for grading
llm_grader = llm.with_structured_output(GradeDocuments)

grade_prompt = ChatPromptTemplate.from_template(
    """You are a grader assessing relevance of a retrieved document to a user question.
    
Retrieved document:
{document}

User question:
{question}

If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
Give a binary score 'yes' or 'no' to indicate whether the document is relevant."""
)

retrieval_grader = grade_prompt | llm_grader

# Example: Grade a document
question = "What is LangChain?"
docs = vectorstore.similarity_search(question, k=1)

grade = retrieval_grader.invoke({
    "question": question,
    "document": docs[0].page_content
})

print(f"Document relevance: {grade.binary_score}")

## 8) Complete CRAG Workflow with LangGraph

Now let's build the full CRAG system as a graph:

```
START → Retrieve → Grade Docs → [Relevant?]
                                  ↓ Yes: Generate Answer → END
                                  ↓ No: Web Search → Generate → END
```

In [ ]:
from langgraph.graph import StateGraph, END, START
from typing import Annotated
import operator

# Define the graph state
class GraphState(TypedDict):
    question: str
    documents: List[Document]
    generation: str
    web_search_needed: bool

# Node 1: Retrieve documents
def retrieve(state: GraphState) -> GraphState:
    question = state["question"]
    documents = vectorstore.similarity_search(question, k=3)
    return {"documents": documents}

# Node 2: Grade documents
def grade_documents(state: GraphState) -> GraphState:
    question = state["question"]
    documents = state["documents"]
    
    filtered_docs = []
    web_search_needed = False
    
    for doc in documents:
        grade = retrieval_grader.invoke({
            "question": question,
            "document": doc.page_content
        })
        
        if grade.binary_score == "yes":
            filtered_docs.append(doc)
        else:
            web_search_needed = True
    
    return {
        "documents": filtered_docs,
        "web_search_needed": web_search_needed
    }

# Node 3: Generate answer
def generate(state: GraphState) -> GraphState:
    question = state["question"]
    documents = state["documents"]
    
    context = "\n\n".join([doc.page_content for doc in documents])
    
    prompt = ChatPromptTemplate.from_template(
        """Answer the question based on the following context:

Context:
{context}

Question: {question}

Answer:"""
    )
    
    chain = prompt | llm | StrOutputParser()
    generation = chain.invoke({"context": context, "question": question})
    
    return {"generation": generation}

# Node 4: Web search (fallback)
def web_search(state: GraphState) -> GraphState:
    print("⚠️ Retrieved documents were not relevant. Would perform web search here.")
    # In production, you'd use a web search tool
    # For this demo, we'll just return a message
    return {"documents": state["documents"]}

# Conditional routing
def decide_to_generate(state: GraphState) -> str:
    if state["web_search_needed"] or len(state["documents"]) == 0:
        return "web_search"
    else:
        return "generate"

# Build the graph
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)
workflow.add_node("web_search", web_search)

# Add edges
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "web_search": "web_search",
        "generate": "generate"
    }
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

# Compile
crag_app = workflow.compile()

# Test it
result = crag_app.invoke({
    "question": "What is LangChain?",
    "documents": [],
    "generation": "",
    "web_search_needed": False
})

print("\n" + "="*80)
print("CRAG Answer:")
print("="*80)
print(result["generation"])

## 📊 When to Use Each Pattern

| Pattern | Use When | Performance Impact |
|---------|----------|-------------------|
| **Query Rewriting** | Users ask vague questions | Low |
| **Multi-Query (RAG Fusion)** | Need comprehensive results | Medium (multiple retrievals) |
| **Contextual Compression** | Documents are long/noisy | Low (saves tokens) |
| **Self-Querying** | Users include filters in queries | Low |
| **Ensemble/Hybrid** | Need both semantic + keyword matching | Medium |
| **Parent Document** | Need context beyond small chunks | Low |
| **CRAG** | Knowledge base has gaps | High (grading + potential web search) |

## 🎯 Production Recommendations

**Start with:**
1. Basic semantic search (embeddings)
2. Add query rewriting for better retrieval
3. Add contextual compression to save tokens

**Scale up with:**
4. Ensemble retrieval (hybrid search)
5. Parent document retriever for better context

**Advanced production:**
6. RAG Fusion for critical queries
7. CRAG with web search fallback

## 📚 Additional Resources

- [LangChain RAG Tutorial](https://python.langchain.com/docs/tutorials/rag/)
- [Advanced RAG Techniques](https://blog.langchain.dev/improving-rag/)
- [CRAG Paper](https://arxiv.org/abs/2401.15884)
- [RAG Evaluation Guide](https://docs.ragas.io/)

## ✅ Key Takeaways

1. **Basic RAG often isn't enough** for production - you need advanced patterns
2. **Query quality matters** - rewriting improves retrieval significantly
3. **Combine techniques** - hybrid search + compression + reranking works best
4. **Evaluate retrieval** - use CRAG patterns to catch poor results
5. **Trade-offs exist** - more sophisticated patterns cost more tokens/latency